In [11]:
import tweepy
import yaml
import pandas as pd

In [9]:
with open('config.yml', "r") as stream:
    try:
        cf = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


auth = tweepy.OAuth1UserHandler(
    cf['twitter_key'], 
    cf['twitter_secret'], 
    cf['twitter_access_key'], 
    cf['twitter_access_secret']
)

In [84]:
client = tweepy.Client(bearer_token=cf['twitter_bearer'])

tweet_fields = ['author_id',
                'attachments',
                'conversation_id',
                'created_at',
                'geo',
                'in_reply_to_user_id',
                'lang',
                'referenced_tweets', # nested
                'context_annotations', # nested
                'entities', # nested
                'public_metrics' # json_normalize]

# @bennstancil
res = client.get_users_tweets(id=1643966166,
                              tweet_fields=tweet_fields)


`get_users_tweets` return a list of `Tweet`. By default, it'll return 10 tweets.

In [194]:
from pandas import json_normalize

# https://stackoverflow.com/questions/70371657/problem-with-getting-tweet-fields-from-twitter-api-2-0-using-tweepy
# Inspired by https://stackoverflow.com/a/34065538/1391964
fields = ['id', 'text'] + tweet_fields
df = pd.DataFrame({fn: getattr(tweet, fn) for fn in fields} for tweet in res.data)
df = pd.concat([df, json_normalize(df['public_metrics'])], axis=1).drop('public_metrics', axis=1)
df = df.set_index('id')
df

,text,author_id,attachments,conversation_id,created_at,geo,in_reply_to_user_id,lang,referenced_tweets,context_annotations,entities,retweet_count,reply_count,like_count,quote_count
id,,,,,,,,,,,,,,,
1505217827921350659,@jillzzy @g_xing https://t.co/u6wkR0dNNM,1643966166,None,1504863910314315789,2022-03-19 16:21:10+00:00,None,1.470270e+07,und,"[(type, id), (type, id)]","[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 0, 'end': 8, 'username...",0,1,2,0
1505029658085150723,@drewwww @g_xing And contrary to Silicon Valle...,1643966166,None,1504863910314315789,2022-03-19 03:53:27+00:00,None,1.643966e+09,en,"[(type, id)]","[{'domain': {'id': '65', 'name': 'Interests an...","{'mentions': [{'start': 0, 'end': 8, 'username...",0,0,0,0
1505027932779073537,@drewwww @g_xing Tesla? I think a more nuanced...,1643966166,None,1504863910314315789,2022-03-19 03:46:35+00:00,None,1.516994e+07,en,"[(type, id)]","[{'domain': {'id': '45', 'name': 'Brand Vertic...","{'mentions': [{'start': 0, 'end': 8, 'username...",0,1,1,0
1504958191704104962,@g_xing I think we fundamentally want to belie...,1643966166,None,1504863910314315789,2022-03-18 23:09:28+00:00,None,1.643966e+09,en,"[(type, id)]",[],"{'mentions': [{'start': 0, 'end': 7, 'username...",0,1,1,0
1504957694809153537,"@g_xing i just saw an ad for the wework movie,...",1643966166,None,1504863910314315789,2022-03-18 23:07:29+00:00,None,5.212854e+08,en,"[(type, id)]","[{'domain': {'id': '123', 'name': 'Ongoing New...","{'mentions': [{'start': 0, 'end': 7, 'username...",0,1,1,0
1504955784429551617,"@ryanjanssen @g_xing @ryanjanssen wait, i'm co...",1643966166,None,1504863910314315789,2022-03-18 22:59:54+00:00,None,1.308685e+08,en,"[(type, id)]",[],"{'mentions': [{'start': 0, 'end': 12, 'usernam...",0,1,0,0
1504904989151571970,here's the thing about this though: it works. ...,1643966166,None,1504904989151571970,2022-03-18 19:38:03+00:00,None,NaN,en,"[(type, id)]",[],"{'urls': [{'start': 235, 'end': 258, 'url': 'h...",0,0,10,1
1504884412596502531,@g_xing to put it another way:\n\n1) company c...,1643966166,None,1504863910314315789,2022-03-18 18:16:18+00:00,None,1.643966e+09,en,"[(type, id)]",[],"{'mentions': [{'start': 0, 'end': 7, 'username...",1,1,2,0
1504883733836488716,"@g_xing true, though I'd argue that that model...",1643966166,None,1504863910314315789,2022-03-18 18:13:36+00:00,None,5.212854e+08,en,"[(type, id)]",[],"{'mentions': [{'start': 0, 'end': 7, 'username...",0,3,1,0


In [175]:
# Context Annotations
df_context_annotations = df[df['context_annotations'].str.len() > 0].loc[:,['id', 'context_annotations']]
df_context_annotations = df_context_annotations.explode('context_annotations').reset_index()
ca = json_normalize(df_context_annotations['context_annotations'], sep="_")
df_context_annotations = pd.concat([df_context_annotations.loc[:, ['id']], ca], axis=1)

df_context_annotations

,id,domain_id,domain_name,domain_description,entity_id,entity_name,entity_description
0,1505217827921350659,10,Person,Named people in the world like Nelson Mandela,982336316460290049,Mark Zuckerberg,Mark Zuckerberg
1,1505029658085150723,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",781974596148793345,Business & finance,NaN
2,1505029658085150723,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",849075881653846016,Startups,Startups
3,1505027932779073537,45,Brand Vertical,Top level entities that describe a Brands indu...,781972125171060736,Auto,NaN
4,1505027932779073537,46,Brand Category,Categories within Brand Verticals that narrow ...,781972125179518977,Auto Manufacturer,NaN
5,1505027932779073537,47,Brand,Brands and Companies,10044199219,Tesla Motors,NaN
6,1505027932779073537,65,Interests and Hobbies Vertical,"Top level interests and hobbies groupings, lik...",847528391163092993,Automotive,Car culture
7,1505027932779073537,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",847528576551337984,Hybrid and electric vehicles,Hybrid and electric vehicles
8,1505027932779073537,66,Interests and Hobbies Category,"A grouping of interests and hobbies entities, ...",847528646185070592,Luxury Cars,Luxury
9,1504957694809153537,123,Ongoing News Story,Ongoing News Stories like 'Brexit',1220701888179359745,COVID-19,NaN


In [236]:
# Entities
df_entities = json_normalize(df['entities']).set_index(df.index)

entity_dict = {}

for entity in df_entities.columns:
    new_df = df_entities.explode(entity)
    new_df = json_normalize(new_df[entity]).set_index(new_df.index).dropna(how='all')
    new_df = new_df.add_prefix(entity + '_')
    entity_dict[entity] = new_df